In [ ]:
# Imports and Setup
import asyncio
import os
import json
import hashlib
from datetime import datetime
from typing import List, Dict, Set
from urllib.parse import urljoin, urlparse

import aiohttp
from bs4 import BeautifulSoup
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field
from agents import Agent, WebSearchTool, Runner
from agents.model_settings import ModelSettings

# Initialize
load_dotenv(override=True)
client = OpenAI()
print("✅ Imports loaded")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports loaded


In [ ]:
# Search Agent Configuration
SEARCH_INSTRUCTIONS = "You are a research assistant. Given a search URL, you search the web for that URL and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
# Planner Agent Configuration
HOW_MANY_SEARCHES = 15

PLANNER_INSTRUCTIONS = f"You are a helpful research assistant. Given a URL, come up with a set of web searches related to that URL \
to collect information to best understand the site. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [ ]:
# Writer Agent Configuration
WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query About website URL. "
    "You will be provided with the original URL, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the Detail report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")


writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [ ]:
# Name Extractor Agent Configuration
NAME_AGENT_INSTRUCTIONS = (
    "You analyze the provided text and extract a single concise name "
    "that best represents the main subject (e.g., site/company/person/product). "
    "If a URL is provided, prefer the name associated with that URL. "
    "Return only the name, no extra words."
)


class NameExtraction(BaseModel):
    name: str = Field(description="The extracted name from the text")


name_extractor = Agent(
    name="NameExtractor",
    instructions=NAME_AGENT_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=NameExtraction,
)

In [ ]:
# Core Research Functions
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output


async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results


async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output


async def extract_name_from_text(text: str, url: str = "") -> str:
    """Extract the name from the report text"""
    prompt = (
        f"Text to analyze:\n{text}\n\n"
        f"Original URL (optional): {url}\n\n"
        "Return only the best fitting name."
    )
    result = await Runner.run(name_extractor, prompt)
    return (result.final_output.name or "").strip()

In [ ]:
# UI Helper Functions
def build_status(percent: float, current_step: int, selected_name: str | None = None, finished: bool = False) -> str:
    """
    Build status text with percentage, steps, and progress bar
    percent: 0–100
    current_step: 0..4 (index of current step)
    finished: True when everything is done
    """
    steps = [
        "Starting research",
        "Planning web searches",
        "Running web searches",
        "Generating report",
        "Extracting name & preparing chatbot",
    ]

    # Progress bar line
    bar_len = 24
    filled = int(bar_len * percent / 100)
    bar = "█" * filled + "░" * (bar_len - filled)

    # Step list with icons
    lines = []
    for i, label in enumerate(steps):
        if finished or i < current_step:
            icon = "✅"
        elif i == current_step:
            icon = "🔄"
        else:
            icon = "⏳"
        lines.append(f"- {icon} Step {i+1}: {label}")

    text = f"### Progress: {percent:.0f}%\n\n`{bar}`\n\n" + "\n".join(lines)

    if finished:
        if selected_name:
            text += f"\n\n**Selected name:** `{selected_name}`"
        text += "\n\n🤖 Chatbot is ready. Ask your questions below."

    return text


# Main Research Pipeline
async def run_full_research(url: str, progress=gr.Progress()):
    """
    Runs the full pipeline for a given URL and returns:
    - status_text (for status Markdown)
    - system_prompt (for chatbot)
    - name (for title, etc.)
    - chatbot_update, msg_update, send_btn_update (to show chatbot only when ready)
    """

    # Step 0: Start
    progress(0.02, desc="Starting research...")
    status_text = build_status(2, current_step=0)

    # Step 1: Plan searches
    progress(0.15, desc="Planning searches")
    status_text = build_status(15, current_step=1)
    search_plan = await plan_searches(url)

    # Step 2: Perform searches
    progress(0.45, desc="Running searches")
    status_text = build_status(45, current_step=2)
    search_results = await perform_searches(search_plan)

    # Step 3: Write report
    progress(0.75, desc="Generating report")
    status_text = build_status(75, current_step=3)
    report = await write_report(url, search_results)

    # Step 4: Extract name
    progress(0.9, desc="Extracting name")
    try:
        raw_name = await extract_name_from_text(report.markdown_report, url)
    except Exception:
        raw_name = ""

    if not raw_name:
        try:
            host = urlparse(url).netloc
            raw_name = host.replace("www.", "") or "the site"
        except Exception:
            raw_name = "the site"

    # Final status, 100%
    progress(1.0, desc="Done")
    status_text = build_status(100, current_step=4, selected_name=raw_name, finished=True)

    # Build context + system prompt
    MAX_SEARCH_ITEMS = 6
    if isinstance(search_results, list):
        search_snippets = search_results[:MAX_SEARCH_ITEMS]
    else:
        search_snippets = [str(search_results)]

    search_context = "\n\n---\n\n".join(str(s)[:800] for s in search_snippets)
    short_summary = getattr(report, "short_summary", "")
    report_excerpt = getattr(report, "markdown_report", "")[:3000]

    system_prompt = (
        "You are an AI assistant representing a specific website.\n\n"
        f"CORE IDENTITY\n"
        f"- Selected name: {raw_name}\n"
        f"- Site URL: {url}\n\n"
        "When the user asks who you are, what your name is, or which site you represent, "
        "you MUST answer using these core facts. Do NOT respond with "
        "\"I'm not sure, I don't have information about that.\" to questions about your own name or identity.\n\n"
        "For all other questions, you MUST rely primarily on the information in the context below "
        "(short summary, web search highlights, and report excerpt).\n\n"
        "If a question is clearly not answerable from this context, or you are not confident that the "
        "information is present, reply exactly:\n"
        "\"I'm not sure, I don't have information about that.\"\n\n"
        "Otherwise, answer concisely and professionally in your own words, based only on this context.\n\n"
        "========== CONTEXT START ==========\n\n"
        f"Short summary:\n{short_summary}\n\n"
        f"Web search highlights:\n{search_context}\n\n"
        f"Report excerpt:\n{report_excerpt}\n\n"
        "========== CONTEXT END ==========\n"
    )

    # Show chatbot only now (after everything succeeded)
    chatbot_update = gr.update(visible=True, value=[], label=f"Chat with {raw_name}")
    msg_update = gr.update(visible=True, value="")
    send_btn_update = gr.update(visible=True)

    return status_text, system_prompt, raw_name, chatbot_update, msg_update, send_btn_update


# Chatbot Functions
def chat_fn(message, history, system_prompt, name):
    """Handle chatbot conversation"""
    # If user tries to chat before running research
    if not system_prompt:
        history = history + [[message, "Please run research with a URL first (above)."]]
        return "", history

    messages = [{"role": "system", "content": system_prompt}]

    # Add chat history
    for user_msg, bot_msg in history:
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if bot_msg:
            messages.append({"role": "assistant", "content": bot_msg})

    # New user message
    messages.append({"role": "user", "content": message})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    answer = response.choices[0].message.content

    history = history + [[message, answer]]
    return "", history


async def handle_run_research(url, progress=gr.Progress()):
    """Handle research button click"""
    if not url or not url.strip():
        return (
            "❗ Please enter a valid URL first.",
            "",
            "the site",
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
        )

    status_text, system_prompt, name, chatbot_update, msg_update, send_btn_update = await run_full_research(
        url, progress=progress
    )

    return status_text, system_prompt, name, chatbot_update, msg_update, send_btn_update


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Chatbot with Agentic AI")

    # URL + button
    with gr.Row():
        url_in = gr.Textbox(label="Site URL", placeholder="https://example.com")
        run_btn = gr.Button("Start Research", variant="primary")

    # Status box
    status_box = gr.Markdown("➡️ Enter a URL and click **Start Research**.")

    # Hidden state: prompt + name
    system_prompt_state = gr.State("")
    name_state = gr.State("the site")

    gr.Markdown("### 💬 Chat with the site")

    # Chatbot area (hidden until research done)
    chatbot = gr.Chatbot(label="Chat", visible=False)
    msg = gr.Textbox(label="Your question", placeholder="Ask something about the site...", visible=False)
    send_btn = gr.Button("Send", visible=False)

    # Handle 'Start Research' click
    run_btn.click(
        fn=handle_run_research,
        inputs=[url_in],
        outputs=[status_box, system_prompt_state, name_state, chatbot, msg, send_btn],
    )

    # Send button click
    send_btn.click(
        fn=chat_fn,
        inputs=[msg, chatbot, system_prompt_state, name_state],
        outputs=[msg, chatbot],
    )

    # Press Enter in textbox
    msg.submit(
        fn=chat_fn,
        inputs=[msg, chatbot, system_prompt_state, name_state],
        outputs=[msg, chatbot],
    )

# Launch Gradio interface
demo.launch(inbrowser=True)

/var/folders/9b/ft46xt2x2yv14x6v95ddt9kw0000gn/T/ipykernel_14976/659641292.py:208: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat", visible=False)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/blocks.py:1994: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  state[block._id] = block.__class__(**kwargs)
